In [2]:
from datasets import load_from_disk
from pprint import pprint
import json

## Load the Dataset

In [3]:
ds = load_from_disk("./dataset")
ds

Dataset({
    features: ['query', 'id', 'answers', 'tools'],
    num_rows: 15
})

In [4]:
example = ds[0]
example

{'query': 'Where can I find live giveaways for beta access and games?',
 'id': 0,
 'answers': '[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]',
 'tools': '[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}]'}

## Data Example

In [5]:
print(f"Query: {example['query']}")
print()
print("Tools:")
pprint(json.loads(example["tools"])[0])
print()
print("Answer:")
pprint(json.loads(example["answers"])[0])

Query: Where can I find live giveaways for beta access and games?

Tools:
{'description': 'Retrieve live giveaways from the GamerPower API based on the '
                'specified type.',
 'name': 'live_giveaways_by_type',
 'parameters': {'type': {'default': 'game',
                         'description': 'The type of giveaways to retrieve '
                                        '(e.g., game, loot, beta).',
                         'type': 'str'}}}

Answer:
{'arguments': {'type': 'beta'}, 'name': 'live_giveaways_by_type'}


**Note: you won't have the "answer" column in our private test set!**

## Example for expected output

In [6]:
example_answer = [{"arguments": {"type": "test_arg"}, "name": "test_tool"}]
example_str = json.dumps(example_answer)
example_str

'[{"arguments": {"type": "test_arg"}, "name": "test_tool"}]'

In [7]:
ds = ds.add_column("my_answers", [example_str for _ in ds["query"]])
ds

Dataset({
    features: ['query', 'id', 'answers', 'tools', 'my_answers'],
    num_rows: 15
})

In [30]:
# This would save the modified dataset to disk
# ds.save_to_disk("./my_dataset")